# 载入数据

In [1]:
import sys
sys.path.append('E:/zlab/')
from gluonx.cifar import Cifar
from gluonx.loader import Transforms
# ---------------------------------------------
import time
import numpy as np
import mxnet as mx

from mxnet import gluon, nd, init
from mxnet import autograd as ag
from mxnet.gluon import nn

from gluoncv.model_zoo import get_model
from gluoncv.utils import makedirs, TrainingHistory

batch_size = 8
cifar = Cifar(batch_size)

C:\ProgramData\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [5]:
class SemiModel(nn.HybridBlock):
    def __init__(self, features, **kwargs):
        super().__init__(**kwargs)
        self.features = features
        self.output = nn.Dense(10)

    def hybrid_forward(self, F, x):
        x = self.features(x)
        return self.output(x)


class TrainX(Transforms):
    def __init__(self, ctx, net, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.__dict__ = self
        self.ctx = ctx
        # 网络预设
        _net = get_model('cifar_resnet20_v1', pretrained=True)
        net.features = _net.features
        net.output.initialize(init.Xavier(magnitude=0.24))
        net.collect_params().reset_ctx(self.ctx)
        net.hybridize()
        self.net = net
        self._opt(self.net)
        
    def get_data(self, batch, aug):
        Xs = nd.stack(*[aug(x)for x in batch[0]]).as_in_context(self.ctx)
        ys = batch[1].as_in_context(self.ctx)
        return Xs, ys

    def test_metric(self, net, val_data):
        metric = mx.metric.Accuracy()
        for i, batch in enumerate(val_data):
            Xs, ys = self.get_data(batch, self.test_aug)
            outputs = net(Xs)
            metric.update(ys, outputs)
        return metric.get()
    
    def _opt(self, net):
        '''
        Opt 初始化
        '''
        # Nesterov accelerated gradient descent
        optimizer = 'nag'
        # Set parameters
        optimizer_params = {'learning_rate': 0.1, 'wd': 0.0001, 'momentum': 0.9}
        # Define our trainer for net
        self.trainer = gluon.Trainer(net.collect_params(), optimizer, optimizer_params)
        self.loss_fn = gluon.loss.SoftmaxCrossEntropyLoss()
        
    def train_op(self, epochs, trainset, valset):
        lr_decay_count = 0
        # Learning rate decay factor
        lr_decay = 0.1
        # Epochs where learning rate decays
        lr_decay_epoch = [80, 160, np.inf]
        train_metric = mx.metric.Accuracy()
        train_history = TrainingHistory(['training-error', 'validation-error'])
        for epoch in range(epochs):
            tic = time.time()
            train_metric.reset()
            train_loss = 0
            # Learning rate decay
            if epoch == lr_decay_epoch[lr_decay_count]:
                self.trainer.set_learning_rate(self.trainer.learning_rate*lr_decay)
                lr_decay_count += 1
            # Loop through each batch of training data
            for i, batch in enumerate(trainset):
                # Extract data and label
                Xs, ys = self.get_data(batch, self.train_aug)
                # AutoGrad
                with ag.record():
                    yhats = self.net(Xs)
                    loss = self.loss_fn(yhats, ys)
                # Backpropagation
                loss.backward()
                # Optimize
                self.trainer.step(trainset.batch_size)
                # Update metrics
                train_loss += loss.sum().asscalar()
                train_metric.update(ys, yhats)
            name, acc = train_metric.get()
            # Evaluate on Validation data
            name, val_acc = self.test_metric(self.net, valset)
            # Update history and print metrics
            train_history.update([1-acc, 1-val_acc])
            print('[Epoch %d] train=%f val=%f loss=%f time: %f' %
                  (epoch, acc, val_acc, train_loss, time.time()-tic))
        # We can plot the metric scores with:
        train_history.plot()

In [6]:
%pylab inline
#pretrain_net = get_model('resnet50_v2')
pretrain_net = get_model('cifar_resnet20_v1')
net = SemiModel(pretrain_net.features)
ctx = mx.gpu(0)
T = TrainX(ctx, net)

Populating the interactive namespace from numpy and matplotlib


C:\ProgramData\Anaconda3\lib\site-packages\mxnet\gluon\block.py:421: UserWarning: load_params is deprecated. Please use load_parameters.
  warnings.warn("load_params is deprecated. Please use load_parameters.")


In [7]:
T.train_op(100, cifar.trainset, cifar.testset)

[Epoch 0] train=0.141000 val=0.150400 loss=2321.988731 time: 170.923959
[Epoch 1] train=0.155000 val=0.184900 loss=2171.827522 time: 167.741445
[Epoch 2] train=0.183000 val=0.155000 loss=2181.532764 time: 175.984311
[Epoch 3] train=0.195000 val=0.181000 loss=2125.135861 time: 172.076050
[Epoch 4] train=0.179000 val=0.204000 loss=2126.414108 time: 168.035015
[Epoch 5] train=0.191000 val=0.165800 loss=2124.126733 time: 172.490623
[Epoch 6] train=0.221000 val=0.191600 loss=2038.952611 time: 168.631577
[Epoch 7] train=0.202000 val=0.169700 loss=2072.118306 time: 169.712934
[Epoch 8] train=0.249000 val=0.170500 loss=1995.646701 time: 168.221692
[Epoch 9] train=0.236000 val=0.171200 loss=1991.790123 time: 167.692465
[Epoch 10] train=0.227000 val=0.142700 loss=2012.541627 time: 169.557547
[Epoch 11] train=0.263000 val=0.184400 loss=2013.327776 time: 168.114671
[Epoch 12] train=0.245000 val=0.190400 loss=1998.488708 time: 167.571918


KeyboardInterrupt: 